In [16]:
#import sys
#sys.path.insert(0,"/Library/Python/2.7/site-packages/") 
#sys.path.insert(0,"/Library/Frameworks/Python.framework/Versions/2.7/bin")

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

In [17]:
X = pd.read_csv("./data/X.csv")
y = pd.read_csv("./data/y.csv")

In [18]:
y_labels = {}
i = 0
for value in y.country_destination.unique():
    if value not in y_labels.keys():
        y_labels[value] = i
        i += 1
    y[y == value] = i-1

print "y is indexed by dict:\n%s" % y_labels

y is indexed by dict:
{'FR': 3, 'NL': 9, 'PT': 8, 'CA': 4, 'DE': 10, 'IT': 7, 'US': 1, 'NDF': 0, 'other': 2, 'AU': 11, 'GB': 5, 'ES': 6}


In [19]:
y = y.country_destination
y.ravel().astype(int)

array([0, 0, 1, ..., 5, 9, 0])

In [20]:
print X.shape
print y.shape

(213466, 150)
(213466,)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [22]:
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(149426, 150)
(149426,)
(64040, 150)
(64040,)


In [ ]:
X_train[X_train.age.isnull()]
X_train.iloc[139805]

## LogisticRegression() yields an accuracy_score of 0.6117 !!!!!

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train, y_train.astype(int))

In [ ]:
y_pred_logreg = logreg.predict(X_test)
print y_pred_logreg.shape
#print type(y_pred)
#print type(y_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_logreg.astype(int))

In [ ]:
conf_matrix_logreg = metrics.confusion_matrix(y_test.astype(int), y_pred_logreg.astype(int))
print conf_matrix_logreg
print sum(sum(conf_matrix_logreg))

## KNN w 90 neighbors yields accuracy 0.6266 
## KNN w 120 neighbors yields accuracy 0.6264

In [ ]:
knn = KNeighborsClassifier(n_neighbors=90)
knn.fit(X_train, y_train.astype(int))
y_pred_knn = knn.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_knn.astype(int))

In [ ]:
conf_matrix_knn = metrics.confusion_matrix(y_test.astype(int), y_pred_knn.astype(int))
print conf_matrix_knn
print sum(sum(conf_matrix_knn))

## We cannot set age NaN's w kneighbors because even 1000 neighbors don't have age attr defined!

In [ ]:
#print X_train.loc[0]
neighbors = knn.kneighbors(X_train.loc[0].reshape(1,-1), 1000)
#print neighbors[1][0]
X_train.iloc[neighbors[1][0]][X_train.age > 0]
#X_train.loc[30956]

### RandomizedSearchCV - start w a small n_iter and increase if necessary: unuseful features are auto eliminated!!!! 

In [ ]:
from sklearn.grid_search import RandomizedSearchCV

k_range = [5,15,30,45,60,75,90,120,150,200,250,300,400,500]
weight_options = ["uniform", "distance"]
param_dist = dict(n_neighbors=k_range, weights=weight_options) # must be a continuous distribution for continuous params!

rand = RandomizedSearchCV(knn, param_dist, cv=10, n_iter=10, scoring='accuracy', n_jobs=-1)
rand.fit(X.astype(int), y.astype(int))
rand.grid_scores_

In [ ]:
print rand.best_score_
print rand.best_params_
print rand.best_estimator_

## NeuralNetwork ( by PyBrain ) yields 0.5998 (w 100,000 samples & 30 epocs) !!
## NeuralNetwork ( by PyBrain ) yields 0.6324 (w 30 epocs & 20,20,20,20,20) 
## NeuralNetwork ( by PyBrain ) yields 0.6348 (w 30 epocs & 100,70,50,30,20) 
## NeuralNetwork ( by PyBrain ) yields 0.6354 (w 60 epocs & 100,70,50,30,20) 

In [ ]:
type(X.shape[0])


In [ ]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure import TanhLayer
from pybrain.structure import SoftmaxLayer
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.utilities import percentError

alldata = ClassificationDataSet(150, 1, nb_classes=12)
for i in range(X.shape[0]):
    alldata.addSample(X[i:i+1].values.tolist()[0], y[i:i+1].values.tolist())

tstdata, trndata = alldata.splitWithProportion( 0.3 )

trndata._convertToOneOfMany( )
tstdata._convertToOneOfMany( )

#net = buildNetwork(150, 100, 50, 12, 1, bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer)
#fnn = buildNetwork( trndata.indim, 20, 20, 20, 20, 20, trndata.outdim, bias=True, hiddenclass=TanhLayer, outclass=SoftmaxLayer ) # yields %45.62 test error
fnn = buildNetwork( trndata.indim, 100, 70, 50, 30, 20, trndata.outdim, outclass=SoftmaxLayer)
                   
#trainer = BackpropTrainer( fnn, dataset=trndata, momentum=0.1, verbose=True, weightdecay=0.01) 
trainer = BackpropTrainer( fnn, dataset=trndata, verbose=True) 

for i in range(30):
    trainer.trainEpochs( 1 )
    trnresult = percentError( trainer.testOnClassData(),
                              trndata['class'] )
    tstresult = percentError( trainer.testOnClassData(
           dataset=tstdata ), tstdata['class'] )

    print "epoch: %4d" % trainer.totalepochs, \
          "  train error: %5.2f%%" % trnresult, \
          "  test error: %5.2f%%" % tstresult    

In [ ]:
y_pred = trainer.testOnClassData(dataset=tstdata)
conf_matrix_fnn = metrics.confusion_matrix(tstdata["class"], y_pred)
print conf_matrix_fnn
print sum(sum(conf_matrix_fnn))
metrics.accuracy_score(tstdata["class"], y_pred)
#tstdata["class"].shape
#trainer.testOnClassData(dataset=tstdata).shape

In [ ]:
for i in range(30):
    trainer.trainEpochs( 1 )
    trnresult = percentError( trainer.testOnClassData(),
                              trndata['class'] )
    tstresult = percentError( trainer.testOnClassData(
           dataset=tstdata ), tstdata['class'] )

    print "epoch: %4d" % trainer.totalepochs, \
          "  train error: %5.2f%%" % trnresult, \
          "  test error: %5.2f%%" % tstresult    

## SVM (C=1.0) accuracy 0.6315

In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train.astype(int))
y_pred = svm.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred.astype(int))

In [ ]:
conf_matrix_svm = metrics.confusion_matrix(y_test.astype(int), y_pred.astype(int))
print conf_matrix_svm
print sum(sum(conf_matrix_svm))

In [ ]:
y_test.value_counts()

## LinearSVC accuracy 0.5873

In [ ]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC()
lsvc.fit(X_train, y_train.astype(int))
y_pred_lsvc = lsvc.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_lsvc.astype(int))

## AdaBoost accuracy (n_estimators=100) 0.6329 !!! (VERY FAST)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abo = AdaBoostClassifier(n_estimators=200)
abo.fit(X_train, y_train.astype(int))
y_pred_abo = abo.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_abo.astype(int))

In [ ]:
conf_matrix_abo = metrics.confusion_matrix(y_test.astype(int), y_pred_abo.astype(int))
print conf_matrix_abo
print sum(sum(conf_matrix_abo))

In [ ]:
predict_proba_test_abo = abo.predict_proba(X_test)
predict_proba_test_abo.shape

In [ ]:
predict_proba_test_abo[:3]

In [ ]:
feat_importances = []
for i in range(len(X_test.columns)):
    #print "%s: %.3f" % (X_test.columns[i], abo.feature_importances_[i])
    feat_importances.append((X_test.columns[i], abo.feature_importances_[i]))

feat_importances = sorted(feat_importances, key=lambda x: x[1], reverse=True)
for pair in feat_importances:
    print pair[0], pair[1]

    

#print X_train.columns
#print abo.feature_importances_

## GradientBoostingClassifier yields accuracy 0.6359

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train.astype(int))
y_pred_gbc = gbc.predict(X_test)

In [ ]:
print metrics.accuracy_score(y_test.astype(int), y_pred_gbc.astype(int))

In [ ]:
conf_matrix_gbc = metrics.confusion_matrix(y_test.astype(int), y_pred_gbc.astype(int))
print conf_matrix_gbc
print sum(sum(conf_matrix_gbc))

In [ ]:
predict_proba_test_gbc = gbc.predict_proba(X_test)
predict_proba_test_gbc.shape

In [ ]:
feat_importances = []
for i in range(len(X_test.columns)):
    #print "%s: %.3f" % (X_test.columns[i], abo.feature_importances_[i])
    feat_importances.append((X_test.columns[i], gbc.feature_importances_[i]))

feat_importances = sorted(feat_importances, key=lambda x: x[1], reverse=True)
for pair in feat_importances:
    print pair[0], pair[1]

## xGBoost yields 0.6368 accuracy! *** BEST ***

In [8]:
import xgboost as xgb

#param = {'bst:max_depth':2, 'bst:eta':1, 'silent':1, 'objective':'binary:logistic' }
param = {'bst:max_depth':2, 'bst:eta':1, 'silent':0, 'objective': 'multi:softmax', 'num_class': 12}
param['nthread'] = 4
#param['eval_metric'] = 'auc'
num_round = 129
dtrain = xgb.DMatrix( X_train, label=y_train )
dtest = xgb.DMatrix( X_test, label=y_test )
evallist  = [(dtest,'eval'), (dtrain,'train')]
#bst = xgb.train( param.items(), dtrain, num_round, evallist, early_stopping_rounds=10)
bst = xgb.train( param.items(), dtrain, num_round, evallist)

[0]	eval-merror:0.369878	train-merror:0.370612
[1]	eval-merror:0.626390	train-merror:0.623339
[2]	eval-merror:0.367583	train-merror:0.368443
[3]	eval-merror:0.367661	train-merror:0.368517
[4]	eval-merror:0.368285	train-merror:0.369226
[5]	eval-merror:0.366849	train-merror:0.367687
[6]	eval-merror:0.366880	train-merror:0.367466
[7]	eval-merror:0.366224	train-merror:0.367138
[8]	eval-merror:0.365100	train-merror:0.365806
[9]	eval-merror:0.364663	train-merror:0.365773
[10]	eval-merror:0.364382	train-merror:0.365512
[11]	eval-merror:0.364475	train-merror:0.365519
[12]	eval-merror:0.364225	train-merror:0.365371
[13]	eval-merror:0.364866	train-merror:0.365566
[14]	eval-merror:0.364413	train-merror:0.365325
[15]	eval-merror:0.364194	train-merror:0.364803
[16]	eval-merror:0.364147	train-merror:0.364629
[17]	eval-merror:0.364179	train-merror:0.364555
[18]	eval-merror:0.364335	train-merror:0.364548
[19]	eval-merror:0.364366	train-merror:0.364414
[20]	eval-merror:0.364350	train-merror:0.364414
[2

In [9]:
y_pred_xgb = bst.predict(xgb.DMatrix(X_test))

In [10]:
print metrics.accuracy_score(y_test.astype(int), y_pred_xgb.astype(int))

0.636758276077


In [11]:
conf_matrix_xgb = metrics.confusion_matrix(y_test.astype(int), y_pred_xgb.astype(int))
print conf_matrix_xgb
print sum(sum(conf_matrix_xgb))

[[31888  5413     6     0     0     0     1     0     0     1     0     0]
 [ 9898  8890     2     0     0     0     0     0     0     0     0     0]
 [ 1685  1300     0     0     0     0     0     0     0     0     0     0]
 [  841   653     1     0     0     0     0     0     0     0     0     0]
 [  232   206     0     0     0     0     0     0     0     0     0     0]
 [  382   333     0     0     0     0     0     0     0     0     0     0]
 [  389   287     0     0     0     0     0     0     0     0     0     0]
 [  490   387     0     0     0     0     1     0     0     0     0     0]
 [   45    20     0     0     0     0     0     0     0     0     0     0]
 [  117   115     0     0     0     0     0     0     0     0     0     0]
 [  148   145     0     0     0     0     0     0     0     0     0     0]
 [   81    83     0     0     0     0     0     0     0     0     0     0]]
64040


In [ ]:
%matplotlib inline
xgb.plot_importance(bst, height=2000)

In [13]:
xgb.to_graphviz(bst, num_trees=2)

ImportError: You must install graphviz to plot tree

## SGDClassifier yields: 0.5824 ! *** WORST *** !

In [25]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train.astype(int))
y_pred_sgd = sgd.predict(X_test)

In [26]:
print metrics.accuracy_score(y_test.astype(int), y_pred_sgd.astype(int))

0.582386008745


In [27]:
conf_matrix_sgd = metrics.confusion_matrix(y_test.astype(int), y_pred_sgd.astype(int))
print conf_matrix_sgd
print sum(sum(conf_matrix_sgd))

[[36801   494     7     0     0     0     0     4     0     1     0     2]
 [18290   495     2     0     0     0     0     1     0     0     0     2]
 [ 2910    75     0     0     0     0     0     0     0     0     0     0]
 [ 1437    57     0     0     0     0     0     0     0     0     0     1]
 [  426    12     0     0     0     0     0     0     0     0     0     0]
 [  693    22     0     0     0     0     0     0     0     0     0     0]
 [  650    26     0     0     0     0     0     0     0     0     0     0]
 [  847    31     0     0     0     0     0     0     0     0     0     0]
 [   64     1     0     0     0     0     0     0     0     0     0     0]
 [  225     7     0     0     0     0     0     0     0     0     0     0]
 [  276    17     0     0     0     0     0     0     0     0     0     0]
 [  163     1     0     0     0     0     0     0     0     0     0     0]]
64040
